# Немного простого кодирования

Для того чтобы размяться, расслабиться и получить ещё немножко удовольствия и очков.

## A. Алгоритм Евклида

[Тот самый](http://e-maxx.ru/algo/export_euclid_algorithm).

Даны $a$ и $b$.

$$
\text{gcd}(a,b) = \begin{cases}
a, & b = 0\\
\text{gcd}(b, a\mod{}b), & b \ne 0
\end{cases}
$$

Реализовать $\text{gcd}$ на Python.



In [1]:
import functools
@functools.lru_cache()
def gcd_recursive(a: 'int',b: 'int') -> 'int':
    if(b==0):
        return a
    return gcd_recursive(b,a%b)
@functools.lru_cache()
def gcd_cyclic(a: 'int',b: 'int') -> 'int':
    if(b>a):
        a,b=b,a
    while(a>0 and b>0):
        d=b
        b=a%b
        a=d
    return a
#ok=True
#for i in range(0,1000):
#    for j in range(0,1000):
#        if(gcd_recursive(i,j)!=gcd_cyclic(i,j)):
#            print(f"Failure on i={i},j={j}")
#            ok=false
#if ok:print("Testing succesful on a,b <= 1000")
#%timeit gcd_recursive(15273,22996)
#%timeit gcd_cyclic(15273,22996)

## B. Реализовать функцию, которая отвечает, простое число, или нет

Т.е. возвращает `True`, если оно простое, и `False` в противном случае.
Можно совершенно «в лоб», с проверкой на делимость, но чтобы проверок было не больше, чем необходимо.

Но поскольку это решение неоптимально, следует воспользоваться возможностями *мемоизации* — декоратором [`@functools.lru_cache`](https://docs.python.org/3/library/functools.html#functools.lru_cache). Он похволяет кешировать результаты функций **без побочных эффектов**, чтобы не считать многократно то, что уже посчитано. При этом при помощи директивы `%timeit` можно посмотреть, насколько мемоизация ускоряет работу функции.

In [2]:
import functools

def is_simple(value: 'int') -> 'bool':
    if(value == 2):
        return True
    if(value % 2 == 0):
        return False
    a=3
    while(a*a <= value):
        if(value % a == 0):
            return False
        a+=2
    return True

@functools.lru_cache()
def is_simple_cached(value: 'int') -> 'bool':
    return is_simple(value)

# На самом деле можно было написать так:
# is_simple_cached = functools.lru_cache()(is_simple)
# Всем понятно, почему две пары скобок?

#print("Простые в первой сотне: "[i for i in range(100) if is_simple_cached(i)])

#%timeit is_simple(500)
#%timeit is_simple_cached(500)
# Впечатляет разница в скорости?
# Не очень - 187 vs 127 ns per loop - моя реализация четные ловит за одно деление с остатком.
#print(is_simple_cached(500))

print(is_simple_cached(6343))#Простое

True


## C. Вывод чисел в системе счисления с произвольным (но не более 36) основанием

Если основание не превосходит $B \le 10$, используются цифры $\{0,1,\ldots, B-1\}$. Если от $10 < B \le 36$, то $\{0,\ldots, 10, A \ldots\}$ в количестве $B$.

Конструктор `int` в Python умеет получать $B$ на вход, что много кого не раз выручало:

In [3]:
print(int('XYZ', 36))

44027


Ещё есть следующие полезные встроенные функции, они выручат нас:

In [4]:
# Функция ord — номер символа в наборе Unicode
print(ord('A'))

# Функция chr — выводит символ с заданным номером:
print(chr(ord('A') + 1))

65
B


А вот такой полезной функции нету. А хочется, чтобы была:

In [7]:
def str_base(value: 'int', base: 'int') -> 'str':
    """Shows value using number base, e.g. str_base(44027, 36)=='XYZ'"""
    #Хотелось бы как-то закэшировать этот словарь, а не собирать его каждый раз...
    if(base<1 or base>36):
        raise NotImplementedError("Функция предназначена для перевода в базы от 1 до 36")
    if(base==1):
        return ''.join(['1']*value)#Определим так, хотя формально это не по условию
    aNum=ord('A')
    symbols=dict()
    for i in range(0,base):
        if(i<=9):
            symbols[i]=i
            continue
        symbols[i]=chr(aNum+(i-10))
    
    n=value
    numString = list()
    while n>0:
        n,mod = divmod(n,base)
        numString.append(str(symbols[mod]))
    numString.reverse()
    return ''.join(numString)

print(str_base(44027, 36))  # хочется увидеть 'XYZ'
print(str_base(423412331,25)+f",{int(str_base(423412331,25),25)==423412331}")
print(str_base(14, 1))#14 единиц

XYZ
1I8N9I6,True
11111111111111


## D. Расширенный Алгоритм Евклида

Тоже [тот самый](http://e-maxx.ru/algo/export_extended_euclid_algorithm).

Даны $a$ и $b$. Обратиться к своим знаниям алгебры (если они не помогли, то можно сходить по ссылке выше) и реализовать $\text{egcd}$, такую что $\text{egcd}(a, b) = \left<x, y, \text{gcd}(a, b)\right>$, такие что $\text{gcd}(a, b) = ax + by$.

In [6]:
import functools
import random
@functools.lru_cache()
def egcd_recursive(a: 'int',b: 'int') -> 'tuple':
    if(b==0):
        return (1,0,a)
    x1,y1,gcd = egcd_recursive(b,a%b)
    #print(f"egcd({a},{b})=({y1-(b//a)*x1},{x1},{gcd})")
    #print(f"egcd({b},{a%b})={x1,y1,gcd}")
    return (y1,x1-y1*(a//b),gcd)
print(egcd_recursive(1,0))
print(egcd_recursive(1312312,31312))
for i in range(1,10):
    a = random.randint(1,100000)
    b = random.randint(1,100000)
    x,y,gcd = egcd_recursive(a,b)
    print(f"egcd({a},{b})=({x},{y},{gcd})")
    if(x*a+y*b!=gcd):
        print(f"Error on a={a},b={b}")


(1, 0, 1)
(1357, -56873, 8)
egcd(66315,43838)=(10331,-15628,1)
egcd(9135,94815)=(-83,8,315)
egcd(99545,27996)=(1697,-6034,1)
egcd(65971,54314)=(-9123,11081,1)
egcd(53424,80031)=(-1143,763,21)
egcd(22784,31313)=(13830,-10063,1)
egcd(2865,42196)=(-4875,331,1)
egcd(298,51783)=(-16508,95,1)
egcd(21162,83394)=(-532,135,6)
